<a href="https://colab.research.google.com/github/Bjegsad/Kontur/blob/main/Kontur_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers
!pip install accelerate
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [2]:
!git config --global user.email 'ya.chepurin1809@gmail.com'
!git config --global user.name 'YuriChep'

# Preprocessing

In [36]:
#Все библиотеки
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import collections
from tqdm.auto import tqdm
import numpy as np
import evaluate
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline
import pandas as pd
import torch
import json

In [28]:
df = load_dataset('json', data_files = '/content/train.json', split = 'train') #загружаем наш датасет
df = df.train_test_split(test_size = 0.3) #делим на трейн и тест выборку
df #посмотрим, что получилось

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-61b7b08461c15aa5/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'extracted_part'],
        num_rows: 1259
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'extracted_part'],
        num_rows: 540
    })
})

In [5]:
train = df['train'] #тренировочная выборка
test = df['test'] #тестовая выборка (в трейновом датасете)


In [6]:
questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in train['label']]
train = train.remove_columns("label").add_column("question", questions)
train[1]
val_questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in test['label']]
test = test.remove_columns("label").add_column("question", val_questions)

Flattening the indices:   0%|          | 0/1259 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/540 [00:00<?, ? examples/s]

In [7]:
#Посмотрим на данные
print("Context: ", train[2]["text"])
print("Question: ", train[2]["question"])
print("Answer: ", train[2]['extracted_part'])

Context:  Приложение 4. ы, подлежащей оплате Застройщику. Обеспечение исполнения обязательств. (если Застройщиком является казенное учреждение настоящий раздел исключается, если в соответствии с законодательством Российской Федерации расчеты по контракту подлежат казначейскому сопровождению, Заказчик вправе исключить настоящий раздел) 3.34. Обязательства Застройщика по исполнению Контракта, обеспечиваются внесением денежных средств (обеспечительный платеж) на указанный в Контракте счет Заказчика или предоставлением независимой гарантии, соответствующей дополнительным требованиям, утвержденным постановлением Правительства Российской Федерации от 08.11.2013 №1005, выданной банком, включенным в предусмотренный частью 1.2 статьи 45 Закона №44-ФЗ перечень банков, отвечающих требованиям, установленным постановлением Правительства от 12.04.2018 №440. Способ обеспечения исполнения Контракта, срок действия независимой гарантии определяются в соответствии с требованиями Закона №44-ФЗ Застройщико

In [8]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru') #https://habr.com/ru/articles/562064/

In [9]:
max_length = 384
stride = 128
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["extracted_part"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_end"][0]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [10]:
train_dataset = train.map(
    preprocess_training_examples,
    batched = True,
    remove_columns=train.column_names,
)
len(train), len(train_dataset)

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

(1259, 2448)

In [11]:
test_dataset = test.map(
    preprocess_validation_examples,
    batched = True,
    remove_columns = train.column_names,
)
len(test), len(test_dataset)

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

(540, 1037)

# Compute Metrics

In [12]:
n_best = 20
max_answer_length = 150
metric = evaluate.load('squad')

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["text"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": str(ex["id"]), "answers": {'text': ex["extracted_part"]['text'], 'answer_start': ex["extracted_part"]['answer_start']}} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

# Training

In [13]:
model = AutoModelForQuestionAnswering.from_pretrained('sberbank-ai/sbert_large_nlu_ru')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#notebook_login() #Hugging Face token

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
args = TrainingArguments(
    "sberbank-ai/sbert_large_nlu_ru_ktr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
)

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/YuryCHep/sbert_large_nlu_ru_ktr into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.726300
1000,0.172200


TrainOutput(global_step=1224, training_loss=0.3794133445016699, metrics={'train_runtime': 1334.7609, 'train_samples_per_second': 7.336, 'train_steps_per_second': 0.917, 'total_flos': 6820424150188032.0, 'train_loss': 0.3794133445016699, 'epoch': 4.0})

In [17]:
tokenizer = AutoTokenizer.from_pretrained('YuryCHep/sbert_large_nlu_ru_ktr')
model = AutoModelForQuestionAnswering.from_pretrained('YuryCHep/sbert_large_nlu_ru_ktr')

In [18]:
predictions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, test_dataset, test)

  0%|          | 0/540 [00:00<?, ?it/s]

{'exact_match': 72.22222222222223, 'f1': 80.56533407321692}

# Prediction

In [31]:
model = AutoModelForQuestionAnswering.from_pretrained('/content/sberbank-ai/sbert_large_nlu_ru_ktr/checkpoint-1224')

In [33]:
test_data = load_dataset('json', data_files = '/content/test.json')['train']

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
#Вставка checkpoint
qa_pipeline = pipeline('question-answering', model = '/content/sberbank-ai/sbert_large_nlu_ru_ktr/checkpoint-1224')

In [34]:
#Делаем предсказания
def predictions(dataset):
  outputs = []
  for ser in dataset:
    label = ser['label']
    question = "Какой фрагмент текста соответствует размеру {}?".format(label)
    context = ser['text']
    answer = qa_pipeline(question = question, context = context)
    output = {'id': ser['id'], 'text': context, 'label': label, 'extracted_part': {'text': [answer['answer']], 'answer_start': [answer['start']], 'answer_end': [answer['end']]}}
    outputs.append(output)
  return outputs

pred = predictions(test_data)

In [37]:
with open('predictions.json', 'w') as f:
  json.dump(pred, f, ensure_ascii = False)